In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# ! pip install requests
# !pip install requests-html

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from requests_html import HTMLSession, AsyncHTMLSession 
import nest_asyncio
import asyncio
from datetime import date, timedelta

In [ ]:
sdate = date(2021,8,1)   # start date
edate = date(2022,2,27)   # end date

In [ ]:
BASE_URL = '/content/drive/MyDrive/AWSHACK/'
PATH = BASE_URL + 'daily-chart-data-after-july.csv'

In [ ]:
def spotify_url(date, region="us"):
  return "https://spotifycharts.com/regional/"+region+"/daily/"+date

In [ ]:
async def call(url):
  nest_asyncio.apply()
  session = AsyncHTMLSession()
  r = session.get(url)
  return await r

In [ ]:
def scrap_daily_data(dates):
  results = []
  for date in dates:
    date = str(date).split(" ")[0]
    url = spotify_url(date)
    coro = call(url)
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(coro)
    html = BeautifulSoup(result.content, 'html.parser')
    table = html.find("table", class_="chart-table")
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for row in rows:
      track = row.find("td", class_="chart-table-track")
      track_name = track.find("strong").text
      artist = track.find("span").text
      streams = row.find("td", class_="chart-table-streams").text
      results.append({
          "artist": artist[3:],
          "track name": track_name,
          "streams": int(streams.replace(',','')),
          "date": date
      })
  return pd.DataFrame.from_records(results)

In [ ]:
dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d')

In [ ]:
df = scrap_daily_data(dates)

In [ ]:
df.to_csv(PATH)